## Projet de deep learning réalisé dans le cadre du module d'IA.

### Sujet choisi : Classification d'images de chats et de chiens en utilisant un réseau neuronal convolutif

#### 1 - Import des librairies nécessaires au projet
- numpy : Utilisé pour gérer les tableaux de données
- pandas : Utilisé pour convertir les données de résultat en csv
- cv2 : Utilisé pour lire et traier les images
- tensorflow : Utilisé pour créer notre réseau de neurone et l'entraîner

In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

#### 2 - Récupération et pré-traitements des données

Ici, on créer une fonction qui parcours toutes les images d'entraînement (dossier "dataset/train/) et plusieurs traitements sont effectués :
- On utilise cv2 pour lire l'image en niveau de gris et la transformer en tableau de pixels (0 à 255)
- Redimensionne le tableau avec un taille de 80 x 80
- Ajoute ce tableau au tableau de données d'entraînement "X".
- Ajoute la catégorie réelle, chien ou chat de l'image au tableau des résultats attendus "y". 

In [23]:
train_path = "./dataset/train/"

X = []
y = []
convert = lambda category : int(category == 'dog')
def create_test_data(train_path):
    for p in os.listdir(train_path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(train_path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)


#### 3 - Création des données d'entraînements
Ici nous appellons la fonction défini précédemment pour créer les données d'entrainements.
Nous donnons ensuite au tableau une forme de vecteur vertical.

In [ ]:
create_test_data(train_path)
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

#### 4 - Normalisation de la valeur des pixels
Avec des valeurs allant de 0 à 255, l'apprentissage pour notre modèle est plus difficile.
C'est pourquoi nous normalisons ces valeurs afin de les ramener entre 0 et 1.

In [16]:
X = X/255.0

#### 5 - Création du modèle
Nous créons un modèle séquentiel à deux couches avec les caractéristiques suivantes :
- 64 filtres
- Un motif de taille (3,3) --> motif de kernel
- Une fonction d'activation relu permettant de remplacer les résultats négatifs par 0

Ces deux couches sont entièrement connectées, c'est-à-dire que chaque neurone est relié à chaque neurone de la couche précédente.

In [20]:
model = Sequential()
# Adds a densely-connectpathed layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### 6 - Entraînment du modèle
- Nous avons choisi d'entraîner notre modèle sur 10 époques. Cet à dire qu'on passe toutes les images d'entraînements 10 fois.
- On décide de passer 32 images par 32 pour réaliser une époque.
- On décide de prendre les derniers 20% des images d'entraînements pour tester notre modèle sur l'époque actuelle.

In [ ]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

#### 7 - Interprétation des résultats

Une fois le modèle entraîné, nous le testons avec les images de tests qui n'ont pas été donné auparavant.
Nous arrondissons les valeurs pour soit obtenir 1 (chien) ou 0 (chat).
Enfin, nous mettons les résultats dans un DataFrame pandas qui sera exporté en csv pour mieux visualiser les résultats.

In [ ]:
predictions = model.predict(X_test)
predicted_val = [int(round(p[0])) for p in predictions]
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})
submission_df.to_csv("submission.csv", index=False)